# AADC Example of supported NumPy operations

In [ ]:
import time
import numpy as np
import scipy.special

### Please uncomment next line if you don't have AADC installed locally

In [11]:
#!pip install https://matlogica.com/DemoReleases/aadc-1.7.5.27-cp3{sys.version_info.minor}-cp3{sys.version_info.minor}-linux_x86_64.whl

In [ ]:
import aadc
import aadc.overrides
from aadc.evaluate_wrappers import evaluate_matrix_inputs
from aadc.recording_ctx import record_kernel

#### Basic workflow consists of:
* Recording, which generates the fast, compiled code (*kernel*), based on executing Python code for a single sample (define-by-run)
* Evaluation, which allows us to evaluate the kernel for more samples in a fast, multi-threaded way

#### To perform recording you need to feed special probing objects, `AADCArrays` into your code.
* Those objects act like numpy arrays.
* We take great care to have as much numpy interop as possible.

In [2]:
rng = np.random.default_rng(1234)
batch_size = 100

# We will record this function, looks like a regular numpy function
a_np = rng.standard_normal((10, 10))
def func(x):
    return (a_np @ x).mean()

# Recording can be done using a context manager. kernel is object which holds the compiled code
with record_kernel() as kernel:
    # You can initialize AADCArrays like numpy arrays
    x = aadc.array(np.ones(10)) 
    x_arg = x.mark_as_input()

    y = func(x)
    y_arg = y.mark_as_output()

# Compute derivatives of y with respect to x
request = [(y_arg, [x_arg])]

# Feed a batch of 100 random vectors as inputs
inputs = [( x_arg, rng.standard_normal((batch_size, 10)) )]

# Evaluate using 4 threads
output_values, output_grads = evaluate_matrix_inputs(kernel, request, inputs, 4)

You are using evaluation version of AADC. Expire date is 20240901


### This workbook will focus on recording, since evaluation is easy and happens completely outside of Python.
* We decided that key to convinient recording is numpy interop
* You can call any numpy function on an AADCArray, and mix the AADCArray with pure numpy arrays. 
* It works intuitively, like in Pandas

In [3]:
arr = aadc.array([1.0, 2.0, 3.0])

with record_kernel() as kernel:
    arr.mark_as_input()

    # Some popular unary functions
    cumsum_arr = np.cumsum(arr)
    print(cumsum_arr) 
    
    cumsum_arr = np.exp(arr)
    print(cumsum_arr) 
    
    cumsum_arr = np.sqrt(arr)
    print(cumsum_arr) 
    
    max_val = np.max(arr)
    print(max_val)
    
    mean_val = np.mean(arr)
    print(mean_val)
    
    prod_val = np.prod(arr)
    print(prod_val)
    
    std_val = np.std(arr)
    print(std_val)
    
    sum_val = np.sum(arr)
    print(sum_val)

    # Scipy ufuncs also work
    sum_val = scipy.special.ndtr(arr)
    print(sum_val)

    # Can mix numpy and AADCArray as needed
    y = np.array([3.0, 4.0, 5.0])
    cov_val = np.cov(arr, y)
    print(cov_val)

aadc.array([idouble([AAD[rv] [adj] :7,1.00e+00]),
       idouble([AAD[rv] [adj] :10,3.00e+00]),
       idouble([AAD[rv] [adj] :11,6.00e+00])], dtype=object)
aadc.array([idouble([AAD[rv] [adj] :14,2.72e+00]),
       idouble([AAD[rv] [adj] :15,7.39e+00]),
       idouble([AAD[rv] [adj] :16,2.01e+01])], dtype=object)
aadc.array([idouble([AAD[rv] [adj] :10,1.00e+00]),
       idouble([AAD[rv] [adj] :6,1.41e+00]),
       idouble([AAD[rv] [adj] :17,1.73e+00])], dtype=object)
idouble([AAD[rv] [adj] :15,3.00e+00])
idouble([AAD[rv] [adj] :14,2.00e+00])
idouble([AAD[rv] [adj] :18,6.00e+00])
idouble([AAD[rv] [adj] :23,8.16e-01])
idouble([AAD[rv] [adj] :22,6.00e+00])
aadc.array([idouble([AAD[rv] [adj] :20,8.41e-01]),
       idouble([AAD[rv] [adj] :11,9.77e-01]),
       idouble([AAD[rv] [adj] :21,9.99e-01])], dtype=object)
aadc.array([[idouble([AAD[rv] [adj] :29,1.00e+00]),
        idouble([AAD[rv] [adj] :30,1.00e+00])],
       [idouble([AAD[rv] [adj] :28,1.00e+00]), 1.0]], dtype=object)
You are usin

### You can also manipulate AADCArrays using Numpy functions (concatenate, stack, expand_dims and so on...)

In [4]:
with record_kernel() as kernel:
    # Broadcast arrays to a common shape
    arr1 = aadc.array([1.0, 2.0])
    arr1.mark_as_input()
    arr2 = aadc.array([[1.0], [2.0]])
    arr1.mark_as_input()
    arr1, arr2 = np.broadcast_arrays(arr1, arr2)
    print(arr1)  # Output: [[1.0, 2.0], [1.0, 2.0]]
    print(arr2)  # Output: [[1.0, 1.0], [2.0, 2.0]]
    
    # Concatenate arrays along an axis
    arr1 = aadc.array([[1.0, 2.0], [3.0, 4.0]])
    arr1.mark_as_input()
    arr2 = aadc.array([[5.0, 6.0]])
    arr1.mark_as_input()
    arr_concat = np.concatenate((arr1, arr2), axis=0)
    print(arr_concat)  # Output: [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]
    
    # Expand the shape of an array
    arr = aadc.array([1.0, 2.0])
    arr.mark_as_input()
    arr_expanded = np.expand_dims(arr, axis=1)
    print(arr_expanded)  # Output: [[1.0], [2.0]]

aadc.array([[idouble([AAD[rv] [adj] :7,1.00e+00]),
        idouble([AAD[rv] [adj] :8,2.00e+00])],
       [idouble([AAD[rv] [adj] :7,1.00e+00]),
        idouble([AAD[rv] [adj] :8,2.00e+00])]], dtype=object)
aadc.array([[1., 1.],
       [2., 2.]])
aadc.array([[idouble([AAD[rv] [adj] :9,1.00e+00]),
        idouble([AAD[rv] [adj] :10,2.00e+00])],
       [idouble([AAD[rv] [adj] :11,3.00e+00]),
        idouble([AAD[rv] [adj] :12,4.00e+00])],
       [5.0, 6.0]], dtype=object)
aadc.array([[idouble([AAD[rv] [adj] :13,1.00e+00])],
       [idouble([AAD[rv] [adj] :14,2.00e+00])]], dtype=object)
You are using evaluation version of AADC. Expire date is 20240901


### Some more advanced functions are also supported, for example interpolation and binary search.
* Note: in the example below we interpolate based on *active inputs* and *active values* on a *fixed* grid
* Example application in finance would be Local Volatility

In [5]:
with record_kernel():
    # Binary search
    x_args = aadc.array([1.0, 2.0, 3.0, 4.0])
    x0_vals = aadc.array([1.1, 2.5, 3.0, 3.9])
    x0_vals.mark_as_input()
    idxs = np.searchsorted(x_args, x0_vals)
    print(idxs)

    # You can index AADCArrays with active types
    print(x_args[idxs])
    
    # Interpolation
    x_args = aadc.array([1.0, 2.0, 3.0, 4.0])
    x0_vals = aadc.array([0.5, 2.5, 3.0, 4.5])
    x0_vals.mark_as_input()
    y_args = aadc.array([1.0, 4.0, 9.0, 16.0])
    y_args.mark_as_input()
    y0_vals = np.interp(x0_vals, x_args, y_args)
    print(y0_vals)

aadc.array([iint(1), iint(2), iint(2), iint(3)], dtype=object)
aadc.array([idouble([AAD[rv] :23,2.00e+00]), idouble([AAD[rv] :24,3.00e+00]),
       idouble([AAD[rv] :25,3.00e+00]), idouble([AAD[rv] :26,4.00e+00])],
      dtype=object)
aadc.array([idouble([AAD[rv] [adj] :36,1.00e+00]),
       idouble([AAD[rv] [adj] :73,6.50e+00]),
       idouble([AAD[rv] [adj] :74,9.00e+00]),
       idouble([AAD[rv] [adj] :75,1.60e+01])], dtype=object)
You are using evaluation version of AADC. Expire date is 20240901


/home/dimach/venv/test/lib/python3.11/site-packages/aadc/ndarray.py:208: RuntimeWarning: invalid value encountered in __truediv__ (vectorized)
  results = method_to_call(*utils.unpack(inputs), out=tup_of_arrays_as_obj(utils.unpack(out)), **kwargs)


### Stochastic if statements
* As we are in the AAD setting, we cannot support recording `if`-based branches if the condition is active
* In a nutshell - whenever branch depends on a kernel input (is not fixed), we have to record both sides
* In cases where output needs to be selected based on a binary condition, you can use `np.where`

In [6]:
with record_kernel():
    # Create some input arrays
    arr1 = aadc.array([-1.0, -2.0, 3.0, 4.0, 5.0])
    arr2 = aadc.array([1.0, 2.0, 3.0, 4.0, 5.0])
    arr3 = aadc.array([3.0, 1.0, 1.0, 1.0, 5.0])
    
    # Mark the input arrays as active
    arr1.mark_as_input()
    arr2.mark_as_input()
    arr3.mark_as_input()
    
    # Define binary conditions
    cond1 = arr1 > 2
    cond2 = arr2 < 4
    cond3 = arr3 != 1
    
    # Combine the conditions using logical operations
    combined_cond = (cond1 & cond2) | cond3
    
    # Use np.where with the combined condition
    result = np.where(combined_cond, arr1 * 2, arr1 / 2)
    print(result)

You are using evaluation version of AADC. Expire date is 20240901
aadc.array([idouble([AAD[rv] [adj] :41,-2.00e+00]),
       idouble([AAD[rv] [adj] :54,-1.00e+00]),
       idouble([AAD[rv] [adj] :55,6.00e+00]),
       idouble([AAD[rv] [adj] :56,2.00e+00]),
       idouble([AAD[rv] [adj] :57,1.00e+01])], dtype=object)


### Controlled monkey-patching
* Recording has a built-in overrides feature which allows you to monkey-patch buried instantiations that would otherwise be problematic to deal with.
* In the example below we try to assign AADCArrays into **slices** of Numpy arrays, which will raise **exceptions**
* Only way to deal with it is to instantiate asset_price_movements as AADCArray, which can be done manually or by requesting overrides (if you really don't want to modify code)

In [12]:
def problematic_code(log_returns):
    asset_price_movements = np.ones((T, assets))
    for t in range(T):
        asset_price_movements[t, :] = asset_price_movements[t - 1, :] * np.exp(log_returns)
    return asset_price_movements

rng = np.random.default_rng(1234)

assets = 10
T = 100

In [13]:
try:
    # Without overrides
    with record_kernel() as kernel:
        log_returns = aadc.array(rng.standard_normal(assets))
        log_returns.mark_as_input()
        asset = problematic_code(log_returns)
except Exception as e:
    print(e)

Cannot convert an active AADCArray to a numpy array
You are using evaluation version of AADC. Expire date is 20240901


In [14]:
# With overrides
with record_kernel(with_overrides=True, wanted_overrides=["np.ones"]) as kernel:
    log_returns = aadc.array(rng.standard_normal(assets))
    log_returns.mark_as_input()
    asset = problematic_code(log_returns)
    print(asset)

You are using evaluation version of AADC. Expire date is 20240901
aadc.array([[idouble([AAD[rv] [adj] :30,5.99e-01]),
        idouble([AAD[rv] [adj] :31,3.76e+00]),
        idouble([AAD[rv] [adj] :32,4.23e-01]),
        idouble([AAD[rv] [adj] :33,1.68e+00]),
        idouble([AAD[rv] [adj] :34,2.82e-01]),
        idouble([AAD[rv] [adj] :35,1.15e-01]),
        idouble([AAD[rv] [adj] :36,1.54e+00]),
        idouble([AAD[rv] [adj] :37,5.66e+00]),
        idouble([AAD[rv] [adj] :38,1.68e+00]),
        idouble([AAD[rv] [adj] :39,3.67e-01])],
       [idouble([AAD[rv] [adj] :29,3.59e-01]),
        idouble([AAD[rv] [adj] :40,1.41e+01]),
        idouble([AAD[rv] [adj] :41,1.79e-01]),
        idouble([AAD[rv] [adj] :42,2.83e+00]),
        idouble([AAD[rv] [adj] :43,7.96e-02]),
        idouble([AAD[rv] [adj] :44,1.33e-02]),
        idouble([AAD[rv] [adj] :45,2.39e+00]),
        idouble([AAD[rv] [adj] :46,3.20e+01]),
        idouble([AAD[rv] [adj] :47,2.83e+00]),
        idouble([AAD[rv] [adj] :48,